# ⚠️⚠️ **WARNING: DO NOT EXECUTE AS-IS** ⚠️⚠️

<h3 style="color:red;">⚠️⚠️ Running the following cells will overwrite previously saved results ⚠️⚠️</h3>

- These operations will **replace** files on disk.
- If unsure, contact Mikaela.
- If something was run by mistake, please contact Mikaela, most files can be recovered for ~1 week from over-writing.

➡️ **Recommended:** Run only after verifying that cache/output directories are altered.

---

# Notebook Summary
This notebook is for loading data from source through filtering stages.

Starts with raw data (species cluster data from GTDB, metadata from GTDB, and metadata from NCBI).

Then filtering stage1 based on minimum clade size and list of NCBI warnings and version status.

Next is stage2 on sourmash for pre-ANI filtering.

Next is stage3, code is both for prepping the fastANI runs and processing based on choosen ANI threshold.

Then cells to prep for mOTUpan (input file and genome pathlist creation)

In [3]:
# Notebook Settings & Imports
#   Note: if you can't load the dataframes please ensure pandas is >v2.0.0, known backcompatibility issues exists
#-------------------------
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
import importlib
import pickle
import os
import re

# Custom functions
import pangenome_utils as pg

# Jupyter settings
#default=last_expr #(https://ipython.readthedocs.io/en/stable/config/options/terminal.html#configtrait-InteractiveShell.ast_node_interactivity)
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Define filters for Stage1 (clade size and NCBI)
#-------------------------
# Minimum clade size is 2, filter out NCBI suppressed OR any of listed warnings

filter_min_clade_size = 2
filter_out_ncbi_version_status = ['suppressed']
filter_out_ncbi_warnings = ['fragmented assembly','genome length too large','genome length too small','hybrid','low quality sequence','misassembled','partial','annotation fails completeness check','low gene count','RefSeq annotation failed','many frameshifted proteins']

# -- STEP 01 -- Load in raw source datasets

Raw unfiltered data (one cell per primary dataframe)

In [7]:
%%time
#-------------------------    
# Load species cluster data from GTDB (this is the base dataset)
#     Output: df_gtdb_species_data_unfiltered
#     Dependencies: None
#-------------------------
# Last runtime: 1.8 s

# Load from file
sp_cluster_file_path = "/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/GTDB_r214/sp_clusters_r214.tsv"
df_gtdb_species_data_unfiltered = pd.read_csv(sp_cluster_file_path, sep='\t')

# Add grouping by ANI
#-------------------------
ASSIGN_ANIGROUP = True
def assign_group(row):
    if row['Mean intra-species ANI'] == 100:
        return 'Group1: 100'
    elif 99.9 >= row['Mean intra-species ANI'] > 99.5:
        return 'Group2: 99.9-99.5'
    elif 99.5 >= row['Mean intra-species ANI'] > 99.0:
        return 'Group3: 99.5-99.0'
    elif 99.0 >= row['Mean intra-species ANI'] > 98.5:
        return 'Group4: 99.0-98.5'
    elif 98.5 >= row['Mean intra-species ANI'] > 98.0:
        return 'Group5: 98.5-98.0'
    elif 98.0 >= row['Mean intra-species ANI'] > 97.0:
        return 'Group6: 98.0-97.0'
    else:
        return 'Group7: LT 97.0'
if ASSIGN_ANIGROUP: df_gtdb_species_data_unfiltered['Group'] = df_gtdb_species_data_unfiltered.apply(assign_group, axis=1)

# Break up the species data into taxonomic columns
temp_df = df_gtdb_species_data_unfiltered['GTDB taxonomy'].str.split(';', expand=True)
temp_df.columns = ['domain', 'phylum', 'class', 'order', 'family', 'genus']
df_gtdb_species_data_unfiltered = pd.concat([df_gtdb_species_data_unfiltered, temp_df], axis=1)

# Add clade_name
df_gtdb_species_data_unfiltered['clade_name']=df_gtdb_species_data_unfiltered['GTDB species']+"--"+df_gtdb_species_data_unfiltered['Representative genome']
df_gtdb_species_data_unfiltered['clade_name'].replace(' ', '_', regex=True, inplace=True)

# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_species_data_unfiltered.pkl', 'wb') as file: pickle.dump(df_gtdb_species_data_unfiltered, file)

CPU times: user 1.5 s, sys: 94.1 ms, total: 1.6 s
Wall time: 1.8 s


In [8]:
%%time
#-------------------------
# Load in all four NCBI report dataframes and concatenate into one
#     Output: df_ncbi_reportdata_unfiltered
#     Dependencies: None
#-------------------------
# Last runtime: 22.9 s

# Read in all four dataframes
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_genbank_historical.txt'
df_ncbi_report_data_genbank_historical = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_genbank.txt'
df_ncbi_report_data_genbank = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_refseq_historical.txt'
df_ncbi_report_data_refseq_historical = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_refseq.txt'
df_ncbi_report_data_refseq = pd.read_csv(tsv_file, delimiter="\t", header=1)

# Concatenate
df_ncbi_reportdata_unfiltered = pd.concat([df_ncbi_report_data_genbank_historical, df_ncbi_report_data_genbank, df_ncbi_report_data_refseq_historical, df_ncbi_report_data_refseq], axis=0, ignore_index=True)
for d in [df_ncbi_report_data_genbank_historical, df_ncbi_report_data_genbank, df_ncbi_report_data_refseq_historical, df_ncbi_report_data_refseq]: del d

# Ensure the "Warnings" column is properly parsed as a list of strings
#df_ncbi_metadata['Warnings'] = df_ncbi_metadata['Warnings'].apply(eval)
df_ncbi_reportdata_unfiltered['excluded_from_refseq'] = df_ncbi_reportdata_unfiltered['excluded_from_refseq'].str.split('; ').apply(lambda x: x if isinstance(x, list) else [x])

# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_ncbi_reportdata_unfiltered.pkl', 'wb') as file: pickle.dump(df_ncbi_reportdata_unfiltered, file)

<timed exec>:16: DtypeWarning: Columns (34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 18.3 s, sys: 3.07 s, total: 21.3 s
Wall time: 22.9 s


In [9]:
%%time
#-------------------------
# Load in all four NCBI report dataframes and concatenate into one
#     Output: df_ncbi_reportdata_unfiltered
#     Dependencies: None
#-------------------------
# Last runtime: 22.9 s

# Read in all four dataframes
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_genbank_historical.txt'
df_ncbi_report_data_genbank_historical = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_genbank.txt'
df_ncbi_report_data_genbank = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_refseq_historical.txt'
df_ncbi_report_data_refseq_historical = pd.read_csv(tsv_file, delimiter="\t", header=1)
tsv_file = '/global/cfs/cdirs/kbase/ke_prototype/mcashman/data/NCBI_datasets/NCBI_reports/assembly_summary_refseq.txt'
df_ncbi_report_data_refseq = pd.read_csv(tsv_file, delimiter="\t", header=1)

# Concatenate
df_ncbi_reportdata_unfiltered = pd.concat([df_ncbi_report_data_genbank_historical, df_ncbi_report_data_genbank, df_ncbi_report_data_refseq_historical, df_ncbi_report_data_refseq], axis=0, ignore_index=True)
for d in [df_ncbi_report_data_genbank_historical, df_ncbi_report_data_genbank, df_ncbi_report_data_refseq_historical, df_ncbi_report_data_refseq]: del d

# Ensure the "Warnings" column is properly parsed as a list of strings
#df_ncbi_metadata['Warnings'] = df_ncbi_metadata['Warnings'].apply(eval)
df_ncbi_reportdata_unfiltered['excluded_from_refseq'] = df_ncbi_reportdata_unfiltered['excluded_from_refseq'].str.split('; ').apply(lambda x: x if isinstance(x, list) else [x])

# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_ncbi_reportdata_unfiltered.pkl', 'wb') as file: pickle.dump(df_ncbi_reportdata_unfiltered, file)

<timed exec>:16: DtypeWarning: Columns (34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 19.3 s, sys: 2.95 s, total: 22.3 s
Wall time: 22.9 s


In [ ]:
#-------------------------    
# Readers for in dataframe created above (toggle as desired)
#-------------------------
print("Dataframes available:")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_species_data_unfiltered.pkl', 'rb') as file: df_gtdb_species_data_unfiltered = pickle.load(file); print("  df_gtdb_species_data_unfiltered")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_ncbi_reportdata_unfiltered.pkl', 'rb') as file: df_ncbi_reportdata_unfiltered = pickle.load(file); print("  df_ncbi_reportdata_unfiltered")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_metadata_unfiltered.pkl', 'rb') as file: df_gtdb_metadata_unfiltered = pickle.load(file); print("  df_gtdb_metadata_unfiltered")

# -- STEP 02 -- Perform Stage1 filtering

## Setup1: Define base filtering dataframe (list of all genomes)

In [ ]:
%%time
#-------------------------
# Create a master filter dataframe to track filters and act as key for applying to dataframes
#     Output: df_master_filter_keys
#     Dependencies: df_gtdb_species_data_unfiltered
#-------------------------
# Last runtime: 694 ms

# Read in source data
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_species_data_unfiltered.pkl', 'rb') as file: df_gtdb_species_data_unfiltered = pickle.load(file)

# Explode species data to get all genomes
df_gtdb_species_data_unfiltered['Clustered genomes'] = df_gtdb_species_data_unfiltered['Clustered genomes'].str.split(',')
df_master_filter_keys = df_gtdb_species_data_unfiltered.explode('Clustered genomes')

# Reorder columns as needed
df_master_filter_keys = df_master_filter_keys[['clade_name', 'Representative genome', 'GTDB species', 'No. clustered genomes', 'GTDB taxonomy', 'Clustered genomes', 'domain', 'phylum', 'class', 'order', 'family', 'genus']]

# Sanity check
if ~(len(df_master_filter_keys) == df_gtdb_species_data_unfiltered['No. clustered genomes'].sum()):
    print("Lengths do not match, something went wrong")
    exit(1)
    
# Reorder and rename columns   
df_master_filter_keys.insert(0, 'genome_ID', df_master_filter_keys.pop('Clustered genomes'))
df_master_filter_keys = df_master_filter_keys.rename(columns={'No. clustered genomes': 'No. clustered genomes in species clade'})

# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'wb') as file: pickle.dump(df_master_filter_keys, file)

## Setup2: Add first stage of filters to master_filter_keys dataframe (overwrites base df)

- FilterA  : clade size
- filterA1 : clade size < 2
- FilterB  : NCBI warnings
- FilterB1 : NCBI warnings defined above

In [11]:
%%time
#-------------------------
# Create filter columns in master filter dataframe (1 is fits filter, 0 is not)
#     Output: df_master_filter_keys (more columns)
#     Dependencies: df_master_filter_keys, df_ncbi_reportdata_unfiltered
#-------------------------
# Last runtime: 8.36 s

# Read in any source data needed
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_ncbi_reportdata_unfiltered.pkl', 'rb') as file: df_ncbi_reportdata_unfiltered = pickle.load(file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file)

#-- Filter based on clade size (FilterA)
df_master_filter_keys['FilterA1'] = np.where(df_master_filter_keys['No. clustered genomes in species clade'] < filter_min_clade_size, 1, 0)

#-- Filter based on NCBI warnings (FilterB)
# Get list of accessions to filter out
accessions_to_filter_out_ncbi = df_ncbi_reportdata_unfiltered[
    ((df_ncbi_reportdata_unfiltered['version_status'].apply(lambda x: x in filter_out_ncbi_version_status))
    | (df_ncbi_reportdata_unfiltered['excluded_from_refseq'].apply(lambda x: any(item in filter_out_ncbi_warnings for item in x))))
]['assembly_accession']
# Cross with dataframe
df_master_filter_keys['FilterB1'] = np.where(df_master_filter_keys['genome_ID'].str[3:].isin(accessions_to_filter_out_ncbi.to_list()), 1, 0)

# Print summary
print(f"Number to filter out w.r.t. clade size: {df_master_filter_keys['FilterA1'].sum()}")
print(f"Number additional to filter out w.r.t. NCBI flags: {df_master_filter_keys[df_master_filter_keys['FilterA1']==0]['FilterB1'].sum()}")
df_master_filter_keys

# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'wb') as file: pickle.dump(df_master_filter_keys, file)

Number to filter out w.r.t. clade size: 56567
Number additional to filter out w.r.t. NCBI flags: 14839
CPU times: user 6.92 s, sys: 1.28 s, total: 8.2 s
Wall time: 8.36 s


## Stage1 Part1 : Filter the species clusters first as any cluster remaining with < the min clade size must be added to the filter_out list

- Adds in FilterC_A1B1 which is anything new to filter out (e.g. FilterC) once Filters A1 and B1 are in place (e.g. clades might be left with only 1 genome)
- Thus updates master_keys df

In [91]:
%%time
#-------------------------
# Filter down raw datasets.  Have to start with gtdb_species as can cause more genomes to be filtered out when clade drops below threshold
#
#     Output: df_gtdb_species_data_filteredStage1, df_master_filter_keys
#     Dependencies: df_gtdb_species_data_unfiltered, df_master_filter_keys
#-------------------------
# Last runtime: 2.03 s

# Load in any dependencies
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_species_data_unfiltered.pkl', 'rb') as file: df_gtdb_species_data_unfiltered = pickle.load(file)

# Get list of genomes IDs to drop
accessions_to_filter_out = df_master_filter_keys[(df_master_filter_keys['FilterA1']==1) | (df_master_filter_keys['FilterB1']==1)]['genome_ID'].to_list()
accessions_to_keep       = df_master_filter_keys[(df_master_filter_keys['FilterA1']==0) & (df_master_filter_keys['FilterB1']==0)]['genome_ID'].to_list()
accessions_to_filter_out_noprefix = [a[3:] for a in accessions_to_filter_out]
accessions_to_keep_noprefix       = [a[3:] for a in accessions_to_keep]


#-------------------------
# Perform the filtering on df_gtdb_metadata_unfiltered
#     Output: df_gtdb_metadata_filteredStage1
#-------------------------
df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_unfiltered.copy()
accessions_to_filter_out_set = set(accessions_to_filter_out)

# Step 1: Convert "Clustered genomes" column to a list
df_gtdb_species_data_filteredStage1["Clustered genomes"] = df_gtdb_species_data_filteredStage1["Clustered genomes"].str.split(",")

# Step 2: Filter the list of genomes in each clade
# Create a log of any genomes to be removed in new column
df_gtdb_species_data_filteredStage1["Clustered genomes removed"] = (
    df_gtdb_species_data_filteredStage1["Clustered genomes"]
    .explode()
    .loc[lambda x: x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)
# Update the "Clustered genomes" column after applying the filters
df_gtdb_species_data_filteredStage1["Clustered genomes"] = (
    df_gtdb_species_data_filteredStage1["Clustered genomes"]
    .explode()
    .loc[lambda x: ~x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)

# Step 3: Calculate the length of the updated "Clustered genomes" list and create a new column, rename accordingly
df_gtdb_species_data_filteredStage1["No. clustered genomes Filtered Stage1"] = df_gtdb_species_data_filteredStage1["Clustered genomes"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_filteredStage1.rename(columns={'No. clustered genomes': 'No. clustered genomes Unfiltered', 'Clustered genomes': 'Clustered genomes retained'})

# Step 4: Filter again w.r.t. size
new_accessions_to_filter_out = df_gtdb_species_data_filteredStage1[df_gtdb_species_data_filteredStage1["No. clustered genomes Filtered Stage1"] < filter_min_clade_size]["Clustered genomes retained"]
filtered_list = [x for x in new_accessions_to_filter_out.to_list() if str(x) != 'nan']
new_accessions_to_filter_out = [item for sublist in filtered_list for item in sublist]
#df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_filteredStage1[df_gtdb_species_data_filteredStage1['Filtered No. clustered genomes'] >= filter_min_clade_size]

# Step 5: Finish filtering the dataframe (drop any < min size) and save
df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_filteredStage1[df_gtdb_species_data_filteredStage1['No. clustered genomes Filtered Stage1'] >= filter_min_clade_size]
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_gtdb_species_data_filteredStage1.pkl', 'wb') as file: pickle.dump(df_gtdb_species_data_filteredStage1, file)

#-------------------------
# Update df_master_filter_keys with any additional genomes that got dropped
#     Output: df_master_filter_keys
#-------------------------
df_master_filter_keys['FilterC_A1B1'] = np.where(df_master_filter_keys['genome_ID'].isin(new_accessions_to_filter_out), 1, 0)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'wb') as file: pickle.dump(df_master_filter_keys, file)

CPU times: user 1.86 s, sys: 160 ms, total: 2.02 s
Wall time: 2.03 s


## Stage1 Part2 : With any added filters in place, filter down reamining raw data sets

In [96]:
%%time
#-------------------------
# Filter down remaining raw datasets
#
#     Output: df_ncbi_reportdata_filteredStage1, df_gtdb_metadata_filteredStage1
#     Dependencies: df_master_filter_keys, df_ncbi_reportdata_unfiltered, df_gtdb_metadata_unfiltered
#-------------------------
# Last runtime: 13.6 s

# Load in any dependencies
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_ncbi_reportdata_unfiltered.pkl', 'rb') as file: df_ncbi_reportdata_unfiltered = pickle.load(file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/unfiltered/df_gtdb_metadata_unfiltered.pkl', 'rb') as file: df_gtdb_metadata_unfiltered = pickle.load(file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file)

# Get list of genomes IDs to drop
accessions_to_filter_out = df_master_filter_keys[(df_master_filter_keys['FilterA1']==1) | (df_master_filter_keys['FilterB1']==1) | (df_master_filter_keys['FilterC_A1B1']==1)]['genome_ID'].to_list()
accessions_to_keep       = df_master_filter_keys[(df_master_filter_keys['FilterA1']==0) & (df_master_filter_keys['FilterB1']==0) & (df_master_filter_keys['FilterC_A1B1']==0)]['genome_ID'].to_list()
accessions_to_filter_out_noprefix = [a[3:] for a in accessions_to_filter_out]
accessions_to_keep_noprefix       = [a[3:] for a in accessions_to_keep]


#-------------------------
# Perform the filtering on df_ncbi_reportdata_unfiltered
#     Output: df_ncbi_reportdata_filteredStage1
#-------------------------
df_ncbi_reportdata_filteredStage1 = df_ncbi_reportdata_unfiltered[df_ncbi_reportdata_unfiltered['assembly_accession'].isin(accessions_to_keep_noprefix)]
if not(len(df_ncbi_reportdata_filteredStage1)==len(accessions_to_keep_noprefix)): print("----!!! WARNING---- NCBI report data missing some genomes to retain") 
# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_ncbi_reportdata_filteredStage1.pkl', 'wb') as file: pickle.dump(df_ncbi_reportdata_filteredStage1, file)


#-------------------------
# Perform the filtering on df_gtdb_metadata_unfiltered
#     Output: df_gtdb_metadata_filteredStage1
#-------------------------
df_gtdb_metadata_filteredStage1 = df_gtdb_metadata_unfiltered[df_gtdb_metadata_unfiltered['accession'].isin(accessions_to_keep)]
if not(len(df_gtdb_metadata_filteredStage1)==len(accessions_to_keep)): print("----!!! WARNING---- GTDB metadata missing some genomes to retain") 
# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_gtdb_metadata_filteredStage1.pkl', 'wb') as file: pickle.dump(df_gtdb_metadata_filteredStage1, file)


CPU times: user 11.1 s, sys: 1.97 s, total: 13.1 s
Wall time: 13.6 s


# -- STEP 03 -- Perform Stage2 filtering - Sourmash filter
Add in an additional Filter marker for the sourmash filtering (currently just on Ecoli)

Modifies:
- Master key dataframe (df_master_filter_keys)
- 3 other dataframes: df_gtdb_species_data_filteredStage2, df_ncbi_reportdata_filteredStage2, df_gtdb_metadata_filteredStage2

In [7]:
%%time
#-------------------------    
# Fetch list of genomes to be removed
#-------------------------
# Last runtime: 6.96 s

# Open master keys
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file)
# Open list of genomes to be removed (already calculated with a sourmash pipeline probably on the GPU nodes)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage2/list_of_removed_genomes_s__Escherichia_coli--RS_GCF_003697165.2_0.9992.txt', 'rb') as file: list_of_removed_genomes_s__Escherichia_coli = [line.strip().decode('utf-8') for line in file]

# Add in a column to the master filtering dataframe
df_master_filter_keys['FilterD1'] = np.where(df_master_filter_keys['genome_ID'].str[3:].isin(list_of_removed_genomes_s__Escherichia_coli), 1, 0)
# Get list of genomes IDs to drop
accessions_to_filter_out = df_master_filter_keys[(df_master_filter_keys['FilterD1']==1)]['genome_ID'].to_list()
print(f"Total number of additional genomes to filter out {len(accessions_to_filter_out)}")

#-------------------------
# Perform the filtering on df_gtdb_metadata_filteredStage1
#     Output: df_gtdb_metadata_filteredStage2
#-------------------------
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_gtdb_species_data_filteredStage1.pkl', 'rb') as file: df_gtdb_species_data_filteredStage1 = pickle.load(file)
df_gtdb_species_data_filteredStage2 = df_gtdb_species_data_filteredStage1.copy()
accessions_to_filter_out_set = set(accessions_to_filter_out)

# Step 1: Filter the list of genomes in each clade
# Create a log of any genomes to be removed in new column
df_gtdb_species_data_filteredStage2["Clustered genomes removed Stage2"] = (
    df_gtdb_species_data_filteredStage2["Clustered genomes retained"]
    .explode()
    .loc[lambda x: x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)
# Update the "Clustered genomes" column after applying the filters
df_gtdb_species_data_filteredStage2["Clustered genomes retained"] = (
    df_gtdb_species_data_filteredStage2["Clustered genomes retained"]
    .explode()
    .loc[lambda x: ~x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)

# Step 2: Calculate the length of the updated "Clustered genomes" list and create a new column, rename accordingly
df_gtdb_species_data_filteredStage2["No. clustered genomes Filtered Stage2"] = df_gtdb_species_data_filteredStage2["Clustered genomes retained"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df_gtdb_species_data_filteredStage2 = df_gtdb_species_data_filteredStage2.rename(columns={'No. clustered genomes': 'No. clustered genomes Filter Stage1', 'Clustered genomes retained': 'Clustered genomes retained Stage2'})

# Step 3: Filter again w.r.t. size
new_accessions_to_filter_out = df_gtdb_species_data_filteredStage2[df_gtdb_species_data_filteredStage2["No. clustered genomes Filtered Stage2"] < filter_min_clade_size]["Clustered genomes retained Stage2"]
filtered_list = [x for x in new_accessions_to_filter_out.to_list() if str(x) != 'nan']
new_accessions_to_filter_out = [item for sublist in filtered_list for item in sublist]
#df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_filteredStage1[df_gtdb_species_data_filteredStage1['Filtered No. clustered genomes'] >= filter_min_clade_size]

# Stop here if any clades dropped below threshold
if len(df_gtdb_species_data_filteredStage2[df_gtdb_species_data_filteredStage2['No. clustered genomes Filtered Stage2']<filter_min_clade_size])>0:
    print(f"----WARNING---- Some clades dropped in size due to Stage 2 filter, please alter code")
    exit(1)

# Step 4: Finish filtering the dataframe (drop any < min size) and save
#df_gtdb_species_data_filteredStage2 = df_gtdb_species_data_filteredStage2[df_gtdb_species_data_filteredStage2['No. clustered genomes Filtered Stage2'] >= filter_min_clade_size]
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_species_data_filteredStage2.pkl', 'wb') as file: pickle.dump(df_gtdb_species_data_filteredStage2, file)

# Print out many rows changed as a sanity check
row_change_count = len(df_gtdb_species_data_filteredStage2[~(df_gtdb_species_data_filteredStage2['No. clustered genomes Filtered Stage1']==df_gtdb_species_data_filteredStage2['No. clustered genomes Filtered Stage2'])])
print(f"Total number of clades changed {row_change_count}")



#-------------------------
# Filter down remaining raw datasets
#
#     Output: df_ncbi_reportdata_filteredStage2, df_gtdb_metadata_filteredStage2
#     Dependencies: df_master_filter_keys, df_ncbi_reportdata_filteredStage1, df_gtdb_metadata_filteredStage1
#-------------------------
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_ncbi_reportdata_filteredStage1.pkl', 'rb') as file: df_ncbi_reportdata_filteredStage1 = pickle.load(file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage1/df_gtdb_metadata_filteredStage1.pkl', 'rb') as file: df_gtdb_metadata_filteredStage1 = pickle.load(file)

# Get list of genomes IDs to drop
accessions_to_filter_out = df_master_filter_keys[(df_master_filter_keys['FilterD1']==1)]['genome_ID'].to_list()
#accessions_to_keep       = df_master_filter_keys[(df_master_filter_keys['FilterD1']==0)]['genome_ID'].to_list()
accessions_to_filter_out_noprefix = [a[3:] for a in accessions_to_filter_out]
#accessions_to_keep_noprefix       = [a[3:] for a in accessions_to_keep]

#-------------------------
# Perform the filtering on df_ncbi_reportdata_filteredStage1
#     Output: df_ncbi_reportdata_filteredStage2
#-------------------------
df_ncbi_reportdata_filteredStage2 = df_ncbi_reportdata_filteredStage1[~df_ncbi_reportdata_filteredStage1['assembly_accession'].isin(accessions_to_filter_out_noprefix)]
#if not(len(df_ncbi_reportdata_filteredStage2)==len(accessions_to_keep_noprefix)): print("----!!! WARNING---- NCBI report data missing some genomes to retain") 
# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_ncbi_reportdata_filteredStage2.pkl', 'wb') as file: pickle.dump(df_ncbi_reportdata_filteredStage2, file)


#-------------------------
# Perform the filtering on df_gtdb_metadata_filteredStage1
#     Output: df_gtdb_metadata_filteredStage2
#-------------------------
df_gtdb_metadata_filteredStage2 = df_gtdb_metadata_filteredStage1[~df_gtdb_metadata_filteredStage1['accession'].isin(accessions_to_filter_out)]
#if not(len(df_gtdb_metadata_filteredStage2)==len(accessions_to_keep)): print("----!!! WARNING---- GTDB metadata missing some genomes to retain") 
# Save dataframe to file (keeping data types)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_metadata_filteredStage2.pkl', 'wb') as file: pickle.dump(df_gtdb_metadata_filteredStage2, file)

print("Done")

Total number of additional genomes to filter out 18003
Total number of clades changed 1
Done
CPU times: user 5.07 s, sys: 1.49 s, total: 6.57 s
Wall time: 6.96 s


# /End/ Dataframe readers

In [ ]:
#-------------------------    
# Readers for any dataframe created above (toggle as desired)
#-------------------------
print("Dataframes available:")
#-- Master Keys
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file); print("  df_master_filter_keys")
#-- Source Data
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_species_data_filteredStage2.pkl', 'rb') as file: df_gtdb_species_data_filteredStage2 = pickle.load(file); print("  df_gtdb_species_data_filteredStage2")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_ncbi_reportdata_filteredStage2.pkl', 'rb') as file: df_ncbi_reportdata_filteredStage2 = pickle.load(file); print("  df_ncbi_reportdata_filteredStage2")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_metadata_filteredStage2.pkl', 'rb') as file: df_gtdb_metadata_filteredStage2 = pickle.load(file); print("  df_gtdb_metadata_filteredStage2")

# -- STEP04 -- Run fastANI


1. Generate genome fna pathlists on NERSC with `dataframes/filtering_stage3/generate_genome_pathlists_fna.py`
    - Input: speies_data dataframe for the list of genomes
    - Output: genomes_pathlist for each clade in user-defined output directory (see filtered_stage3)

2. Run fastANI in `/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/pairwise_ANI/species_level_runs`
    - Previously run on both NERSC and KBase GPU (58 largest plus 2 stragglers)
    - Now all result files back on NERSC in that dir in RESULTS/MERGED_OUT

3. Process results (two stages)
    - STEP01 process the results per_clade `/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP01_process_fastANI` 
    - STEP02 process the per_clade results per_genomepair `/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI`

In [32]:
#-------------------------    
# Readers for any dataframe created above (toggle as desired)
#-------------------------
print(">>> Dataframes available (fastANI results Per Clade):")
for x in ['df_fastANI_byClade_LT500']: print(x)
#-- fastANI results Per Clade
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP01_process_fastANI/df_fastANI_byClade_LT500.pkl', 'rb') as file: df_fastANI_byClade_LT500 = pickle.load(file); print("  df_fastANI_byClade_LT500")

print("\n>>> Dataframes available (fastANI results Per Genome Pair):")
for x in ['df_fastANI_byGenomePair_LT500','df_fastANI_byGenomePair_combinedGTE500sm','df_fastANI_byGenomePair_SE','df_fastANI_byGenomePair_KP','df_fastANI_byGenomePair_SA','df_fastANI_byGenomePair_EC']: print(x)
#-- fastANI results Per Genome Pair
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_LT500.pkl', 'rb') as file: df_fastANI_byGenomePair_LT500 = pickle.load(file); print("  df_fastANI_byGenomePair_LT500")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_combinedGTE500sm.pkl', 'rb') as file: df_fastANI_byGenomePair_combinedGTE500sm = pickle.load(file); print("  df_fastANI_byGenomePair_combinedGTE500sm")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_SE.pkl', 'rb') as file: df_fastANI_byGenomePair_SE = pickle.load(file); print("  df_fastANI_byGenomePair_SE")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_KP.pkl', 'rb') as file: df_fastANI_byGenomePair_KP = pickle.load(file); print("  df_fastANI_byGenomePair_KP")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_SA.pkl', 'rb') as file: df_fastANI_byGenomePair_SA = pickle.load(file); print("  df_fastANI_byGenomePair_SA")
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/RESULTS/df_fastANI_byGenomePair_EC.pkl', 'rb') as file: df_fastANI_byGenomePair_EC = pickle.load(file); print("  df_fastANI_byGenomePair_EC")
print("\n>>> Complete Dataframes available (fastANI results Per Genome Pair):")
print('df_fastANI_byGenomePair_combinedALL')
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/df_fastANI_byGenomePair_combinedALL.pkl', 'rb') as file: df_fastANI_byGenomePair_combinedALL = pickle.load(file); print("  df_fastANI_byGenomePair_combinedALL")

>>> Dataframes available (fastANI results Per Clade):
df_fastANI_byClade_LT500

>>> Dataframes available (fastANI results Per Genome Pair):
df_fastANI_byGenomePair_LT500
df_fastANI_byGenomePair_combinedGTE500sm
df_fastANI_byGenomePair_SE
df_fastANI_byGenomePair_KP
df_fastANI_byGenomePair_SA
df_fastANI_byGenomePair_EC

>>> Complete Dataframes available (fastANI results Per Genome Pair):
df_fastANI_byGenomePair_combinedALL


# -- STEP 05 -- Make master geneome-pairs dataframe for filtering down fastANI output

fastANI was run on some clades pre-filtering.  This means we need to filter down the list before we process it and update the species dataframe.  

## Part 1: Filter down original set of fastANI results

In [2]:
%%time
#-------------------------    
# Starting with the data from Stage2, generate all valid genome pairs
#     Note: Shouldn't crash notebook now that Stage2 exists, can also do in batches (see commented out code below)
#-------------------------
#     Input: df_gtdb_species_data_filteredStage2
#     Output: valid_geneone_pairs_all
#-------------------------
# Last runtime: 4min 56s
from itertools import combinations

with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_species_data_filteredStage2.pkl', 'rb') as file: df_gtdb_species_data_filteredStage2 = pickle.load(file)

#-- Previous verison: Had to do this in batches with the top 4 largest seperated as they crashed the notebook
#df = df_gtdb_species_data_filteredStage2[4:].copy()  #Set05 (everything else)
#df = df_gtdb_species_data_filteredStage2[3:4].copy() #Set04 (s__Salmonella_enterica--RS_GCF_000006945.2 - 11456 genomes)
#df = df_gtdb_species_data_filteredStage2[2:3].copy() #Set03 (s__Staphylococcus_aureus - 14641 genomes)
#df = df_gtdb_species_data_filteredStage2[1:2].copy() #Set02 (s__Klebsiella_pneumoniae - 14338 genomes)
#df = df_gtdb_species_data_filteredStage2[0:1].copy() #Set01 (EC)
#-- New version: can handle all data
df = df_gtdb_species_data_filteredStage2

# Function to generate pairwise combinations
def pairwise_combinations(item_list):
    return [tuple(sorted(pair)) for pair in combinations(item_list, 2)]

# Apply the function to the 'Items' column
df['Pairwise'] = df['Clustered genomes retained Stage2'].apply(pairwise_combinations)

# Concatenate all pairwise combinations into a single list
result = [pair for pairs in df['Pairwise'] for pair in pairs]
del df
print(f"Total number of valid genome pairs: {len(result)}")

# Save to a file
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/valid_geneone_pairs_all_atStage2.pkl', 'wb') as file:
    pickle.dump(result, file)
del result

step01
step02
step03
CPU times: user 4min 18s, sys: 37.6 s, total: 4min 55s
Wall time: 4min 56s


In [34]:
%%time
#-------------------------    
# Validate the size of the valid genome-pair list is right by doing the math on the species clusters dataframe
#     Input: df_gtdb_species_data_filteredStage2, valid_geneone_pairs_all
#     Output: n/a (just prints)
#-------------------------
# Last runtime: 1min 

#--  
# Compute how many pairs are to be expected (post-filtering Stage2) for a specific clade only
#   Note: Selfs are not included as final genepair dataframe only uses the txt_ column
#--
from math import comb

with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_species_data_filteredStage2.pkl', 'rb') as file: df_gtdb_species_data_filteredStage2 = pickle.load(file)
df = df_gtdb_species_data_filteredStage2

# Function to calculate total pairwise combinations for a list
def total_pairwise_combinations(item_list):
    n = item_list
    return comb(n+1, 2) - n #all pairs w/o r.t. order and no selfs

# Apply the function to the 'Items' column and create a new 'Pairwise_Total' column
df['Pairwise_Total'] = df['No. clustered genomes Filtered Stage2'].apply(total_pairwise_combinations)

# Sum the total pairwise combinations
total_sum = df['Pairwise_Total'].sum()

print(f"Total Pairwise Combinations according to species cluster file: {total_sum:,}")


# Get length of valid pairs calculated above
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/valid_geneone_pairs_all_atStage2.pkl', 'rb') as file: valid_geneone_pairs_all = pickle.load(file)
number_of_marked_valid_pairs = len(valid_geneone_pairs_all)
del valid_geneone_pairs_all
print(f"Total Pairwise Combinations according to valid pairs list:     {number_of_marked_valid_pairs:,}")


#--  
# Compare
#--
if total_sum == number_of_marked_valid_pairs: print("\nValue match, the world is good\n")
else: print(f"\nThe counts are off by {total_sum-number_of_marked_valid_pairs}, the world is not good\n")

Total Pairwise Combinations according to species cluster file: 537,061,532
Total Pairwise Combinations according to valid pairs list:     537,061,532

Value match, the world is good

CPU times: user 38.8 s, sys: 19.4 s, total: 58.1 s
Wall time: 1min


In [36]:
#-------------------------    
# Optional! For inspection only
#
# Compute how many pairs are to be expected (post-filtering Stage2) for a specific clade only (if needed)
#   Note: Selfs are not included as final genepair dataframe only uses the txt_ column
#-------------------------
from math import comb

CLADE_OF_INTEREST='s__UBA184_sp021817245--GB_GCA_021817245.1'
#--------

df = df_gtdb_species_data_filteredStage2[df_gtdb_species_data_filteredStage2['clade_name']==CLADE_OF_INTEREST]

# Function to calculate total pairwise combinations for a list
def total_pairwise_combinations(item_list):
    n = item_list
    return comb(n+1, 2) - n #all pairs w/o r.t. order and no selfs

# Apply the function to the 'Items' column and create a new 'Pairwise_Total' column
df['Pairwise_Total'] = df['No. clustered genomes Filtered Stage2'].apply(total_pairwise_combinations)

# Sum the total pairwise combinations
total_sum = df['Pairwise_Total'].sum()

print("Total Pairwise Combinations:", total_sum)

Total Pairwise Combinations: 3


/tmp/ipykernel_2356682/1525016944.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pairwise_Total'] = df['No. clustered genomes Filtered Stage2'].apply(total_pairwise_combinations)


In [ ]:
%%time
# NEW WIP
#-------------------------    
# Filter down fastANI results and validate counts
#     Input: df_genepairs_ANIvAF, valid_geneone_pairs_all_atStage2
#     Output: df_geneomepairs_ANIvAF
#-------------------------
# Last runtime: 

# Read in the original results (no filtering applied) and make a key column to merge with the valid pairs list
# TODO---- update file name and path (df_genepairs_ANIvAF)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/STEP02_formatting_fastANI/df_fastANI_byGenomePair_combinedALL.pkl', 'rb') as file: df_genepairs_ANIvAF = pickle.load(file)
df_genepairs_ANIvAF['key']= df_genepairs_ANIvAF[['Genome1', 'Genome2']].apply(lambda x: '_'.join(sorted([col for col in x])), axis=1)

# Merge the valid pairs with the original data to perform filtering
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/valid_geneone_pairs_all_atStage2.pkl', 'rb') as file: valid_geneone_pairs_all = pickle.load(file)
## Or Read in the valid genome pairs for the orignal set only
#with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/temp/df_valid_genepairs_originalSet_non58.pkl', 'rb') as file: df_valid_genepairs_originalSet_non58 = pickle.load(file)

# Merge
df_merged = valid_geneone_pairs_all.merge(df_genepairs_ANIvAF, on='key', how='left')

# Check for missing values
# TODO---- Is this the only check I need to make sure I have all the information I need?  I think so...
missing_matches = df_merged[df_merged['column_from_df_genepairs_ANIvAF'].isna()]
if missing_matches.empty:
    print("All rows from valid_geneone_pairs_all found a match in df_genepairs_ANIvAF")
else:
    print("There are missing matches. Check the missing_matches dataframe for details.")
    print(missing_matches)

# Save to a file
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/df_fastANI_byGenomePair_combinedALL_ValidOnly_atStage2_ANIvAF.pkl', 'wb') as file: pickle.dump(df_merged, file)

## Part2 : Create Stage3 filtered species_data dataframe

Was running this on NERSC with both sets of genenome pair information (see `/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/filter_down_species_cluster_all_clades.py`).

Thus the below is outdated, but can be used to test new filter limits.

In [27]:
# This currently only runs on the originalSet, see NERSC for running on both sets (all data)

#-------------------------    
# Setup dataframe to iterate on
#     Input: df_original_genepairs_ANIvAF_Stage2Filtered
#     Output: df_gtdb_species_data_filteredStage3 (more columns)
#     Dependencies: df_gtdb_metadata_filteredStage2 (for clade name), df_master_filter_keys
#-------------------------
# Last runtime: quick

# Load in a genome pair dataframe to work off of
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/df_original_genepairs_ANIvAF_Stage2Filtered_saved_redosAdded.pkl', 'rb') as file: df_original_genepairs_ANIvAF_Stage2Filtered_saved_redosAdded = pickle.load(file)
df = df_original_genepairs_ANIvAF_Stage2Filtered_saved_redosAdded.copy()
del df_original_genepairs_ANIvAF_Stage2Filtered_saved_redosAdded

# Set the filter
FILTER=99.9999
df = df[df['ANI']>=FILTER]

# Add in clade_names and genome size to reference df
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_metadata_filteredStage2.pkl', 'rb') as file: df_gtdb_metadata_filteredStage2 = pickle.load(file)
df_gtdb_metadata_filteredStage2['clade_name']=df_gtdb_metadata_filteredStage2['gtdb_taxonomy_species']+"--"+df_gtdb_metadata_filteredStage2['gtdb_genome_representative']
df_gtdb_metadata_filteredStage2['clade_name'].replace(' ', '_', regex=True, inplace=True)
df_gtdb_metadata_filteredStage2=df_gtdb_metadata_filteredStage2[['accession','clade_name','genome_size']]
df_gtdb_metadata_filteredStage2['accession']=df_gtdb_metadata_filteredStage2['accession'].str[3:]

# Add in clade_names and genome size to df
# Merge Genome1 information & clade_name
df = pd.merge(df, df_gtdb_metadata_filteredStage2[['accession', 'clade_name', 'genome_size']], how='left', left_on='Genome1', right_on='accession')
df = df.rename(columns={'genome_size': 'Genome1_size'})
# Merge Genome2 information
df = pd.merge(df, df_gtdb_metadata_filteredStage2[['accession', 'genome_size']], how='left', left_on='Genome2', right_on='accession')
df = df.rename(columns={'genome_size': 'Genome2_size'})
# Clean up
df = df.drop(['accession_x', 'accession_y'], axis=1, errors='ignore')

# Make a new column for the min size (by row/axis=1)
df['min_size']=df[['Genome1_size', 'Genome2_size']].min(axis=1)

# Print stuff
print(f"Number of rows to be processed: {len(df)}")
print("Sample of dataframe:")
df[:4]


#-------------------------    
# Perform filtering of individual genomes
#-------------------------
# Last runtime: 

# Setup
set_of_removed_genomes = set()
count_number_of_already_removed=0
df.sort_values(by='min_size', inplace=True, ignore_index=True)


# Iterate
for index, row in df.iterrows():
    #if index%10000==0: print(index)
    if row['Genome1'] in set_of_removed_genomes or row['Genome2'] in set_of_removed_genomes:
        count_number_of_already_removed+=1
    if row['Genome1_size']<row['Genome2_size']:
        set_of_removed_genomes.add(row['Genome1'])
    else:
        set_of_removed_genomes.add(row['Genome2'])
        
# Finish up
print(f"Total number of removed genomes: {len(set_of_removed_genomes)}")
print(f"Total number of times a genome was already removed: {count_number_of_already_removed}")

# Remove genomes from species data dataframe to see how many clades are lost


#-------------------------    
# Clades removed
#-------------------------
# ---- Part 1, update master keys dataframe
# Read in any source data needed
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/df_master_filter_keys.pkl', 'rb') as file: df_master_filter_keys = pickle.load(file)
#-- Filter based on clade size (FilterE)
df_master_filter_keys['FilterE1'] = np.where(df_master_filter_keys['genome_ID'].str[3:].isin(list(set_of_removed_genomes)), 1, 0)

# ---- Part 2, merge with species dataframe
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_species_data_filteredStage2.pkl', 'rb') as file: df_gtdb_species_data_filteredStage2 = pickle.load(file)
df_gtdb_species_data_filteredStage3 = df_gtdb_species_data_filteredStage2.copy()
del df_gtdb_species_data_filteredStage2
accessions_to_filter_out_set = set(df_master_filter_keys[(df_master_filter_keys['FilterE1']==1)]['genome_ID'].to_list())

# Step 1: Filter the list of genomes in each clade
# Create a log of any genomes to be removed in new column
df_gtdb_species_data_filteredStage3["Clustered genomes removed Stage3"] = (
    df_gtdb_species_data_filteredStage3["Clustered genomes retained Stage2"]
    .explode()
    .loc[lambda x: x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)
# Update the "Clustered genomes" column after applying the filters
df_gtdb_species_data_filteredStage3["Clustered genomes retained"] = (
    df_gtdb_species_data_filteredStage3["Clustered genomes retained Stage2"]
    .explode()
    .loc[lambda x: ~x.isin(accessions_to_filter_out_set)]
    .groupby(level=0)
    .agg(list)
)

# Step 2: Calculate the length of the updated "Clustered genomes" list and create a new column, rename accordingly
df_gtdb_species_data_filteredStage3["No. clustered genomes Filtered Stage3"] = df_gtdb_species_data_filteredStage3["Clustered genomes retained"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df_gtdb_species_data_filteredStage3 = df_gtdb_species_data_filteredStage3.rename(columns={'No. clustered genomes': 'No. clustered genomes Filter Stage2', 'Clustered genomes retained': 'Clustered genomes retained Stage3'})

# Step 3: Filter again w.r.t. size
new_accessions_to_filter_out = df_gtdb_species_data_filteredStage3[df_gtdb_species_data_filteredStage3["No. clustered genomes Filtered Stage1"] < filter_min_clade_size]["Clustered genomes retained Stage3"]
filtered_list = [x for x in new_accessions_to_filter_out.to_list() if str(x) != 'nan']
new_accessions_to_filter_out = [item for sublist in filtered_list for item in sublist]
#df_gtdb_species_data_filteredStage1 = df_gtdb_species_data_filteredStage1[df_gtdb_species_data_filteredStage1['Filtered No. clustered genomes'] >= filter_min_clade_size]

# Length Check
clades_dropped=len(df_gtdb_species_data_filteredStage3[df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage3']<filter_min_clade_size])
print(f"Total number of clades dropped {clades_dropped}")
    
# Step 4: Finish filtering the dataframe (drop any < min size) and save
df_gtdb_species_data_filteredStage3 = df_gtdb_species_data_filteredStage3[df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage3'] >= filter_min_clade_size]

# Save files
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/df_gtdb_species_data_filteredStage3.pkl', 'wb') as file: pickle.dump(df_gtdb_species_data_filteredStage3, file)
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/df_master_filter_keys_filteredStage3.pkl', 'wb') as file: pickle.dump(df_master_filter_keys, file)

# Print out many rows changed as a sanity check
row_change_count = len(df_gtdb_species_data_filteredStage3[~(df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage2']==df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage3'])])
print(f"Total number of clades changed {row_change_count}")

Number of rows to be processed: 14953
Sample of dataframe:


,Genome1,Genome2,ANI,AF,clade_name,Genome1_size,Genome2_size,min_size
0,GCA_902606795.1,GCA_003209225.1,100.0,0.989899,s__AAA240-E13_sp003209225--GB_GCA_003209225.1,966640,966640,966640
1,GCA_902608355.1,GCA_003211695.1,100.0,1.000000,s__AAA240-E13_sp003211695--GB_GCA_003211695.1,1138562,1138562,1138562
2,GCA_902528115.1,GCA_003282145.1,100.0,0.996947,s__AAA536-G10_sp003282145--GB_GCA_003282145.1,2044868,2044868,2044868
3,GCA_902760045.1,GCA_900320825.1,100.0,0.981994,s__AC2028_sp900320825--GB_GCA_900320825.1,3029516,3029516,3029516


Total number of removed genomes: 6794
Total number of times a genome was already removed: 8180
Total number of clades dropped 779


# -- STEP 06 -- Setup & Run mOTUpan

## Part 1 - Generate clade lists for mOTUpan runs

In [ ]:
#-------------------------    
# Generate clade list files to run mOTUpan
#     Input: df_original_genepairs_ANIvAF_Stage2Filtered
#     Output: df_gtdb_species_data_filteredStage3 (more columns)
#     Dependencies: df_gtdb_metadata_filteredStage2 (for clade name), df_master_filter_keys
#-------------------------
# Last runtime: quick

with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/df_gtdb_species_data_filteredStage3_99pt9ANIthresh.pkl', 'rb') as file: df_gtdb_species_data_filteredStage3 = pickle.load(file)

# Split into two files for batching (clade sizes less than 500 and clade sizes greater than or equal to 500)
dir_out = "/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/"
file_out_LT = "clade_list_xANIthresh_sizeAllLT500.txt"
file_out_GT = "clade_list_xANIthresh_sizeGTE500.txt"

# Sort by size so the smaller runs finish first
df_gtdb_species_data_filteredStage3=df_gtdb_species_data_filteredStage3.sort_values('No. clustered genomes Filtered Stage3')

# Get list of clade names and distribute
clade_list_LT = df_gtdb_species_data_filteredStage3[df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage3'] < 500]['clade_name'].tolist()
clade_list_GT = df_gtdb_species_data_filteredStage3[df_gtdb_species_data_filteredStage3['No. clustered genomes Filtered Stage3'] >= 500]['clade_name'].tolist()
print(f"Clades less than 500 in size: {len(clade_list_LT)}")
clade_list_LT[:2]
print(f"\nClades greater than or equal to 500 in size: {len(clade_list_GT)}")
clade_list_GT[:2]

# Save both lists
with open(dir_out+file_out_LT, 'w') as file: 
    for item in clade_list_LT: file.write(f"{item}\n")
with open(dir_out+file_out_GT, 'w') as file: 
    for item in clade_list_GT: file.write(f"{item}\n")

## Part2 - Generate faa pathlists for each clade

Also running on NERSC (see generate_genome_pathlists_faa.py in /filtering_stage3)

Note taking too long on interactive nodes (not done in 4 hours, but can get to run in 15 min on login node).

In [131]:
#-------------------------    
# Generate pathlists to faa files
#     Input: df_gtdb_species_data_filteredStage3
#     Output: clade_path_lists_filtered_stage3/genome_pathlist_CLADE.txt
# Note: copied from script on NERSC
#-------------------------

print("WARNING: This cell will exit immediately to prevent accidental execution - unsure if jup can handle it, please run on login node for ~15 min execution.")
raise KeyboardInterrupt("Cell execution stopped by user.")

with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/df_gtdb_species_data_filteredStage3.pkl', 'rb') as file: df_gtdb_species_data_filtered = pickle.load(file)


# Generate pathlists (paths to fnas for a clade) for a LIST of clades
#-------------------------

# Get list of clades to generate pathlists for using desired criteria 
#to_gen=df_gtdb_species_data_filtered[(df_gtdb_species_data_filtered['Filtered No. clustered genomes']>500) & (df_gtdb_species_data_filtered['Filtered No. clustered genomes']<32000)]['clade_name'].to_list()
to_gen=df_gtdb_species_data_filtered[:]['clade_name'].to_list()

# Iterate through each clade
for clade in to_gen:
    #print(clade)
    
    # Custom do a single one (e.g. need to drop a genome)
    clade_name=clade #"s__Escherichia_coli--RS_GCF_003697165.2"
    outpath="/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/clade_path_lists_filtered_stage3"
    base_directory = "/global/cfs/cdirs/kbase/jungbluth/Projects/Project_Pangenome_GTDB/GTDB_r214_by_spcluster/"
    #base_directory_out = "GTDB_v214_download/ftp.ncbi.nlm.nih.gov/genomes/all/"
    row=df_gtdb_species_data_filtered[df_gtdb_species_data_filtered['clade_name'] == clade_name]

    outfile = outpath + "/genome_pathlist_" + clade_name + ".txt"
    genomes = row['Clustered genomes retained Stage3'].iloc[0]
    if ~(len(genomes) == row['No. clustered genomes Filtered Stage3'].iloc[0]):
        print("Something went wrong")
        exit(1)
    paths = []

    for genome in genomes:
        genome = genome[3:]
        #print(genome)

        # Construct the subdirectory path based on the given string
        subdirectory_path = os.path.join(base_directory, clade)

        # Initialize a variable to count matching files
        matching_file_count = 0
        matching_file_path = None

        # Iterate through subdirectories and look for .fna.gz files
        for file in os.listdir(subdirectory_path):
            if file.endswith(".faa.gz") and genome in file:
                matching_file_count += 1
                matching_file_path = os.path.join(base_directory, file)
                #print(matching_file_path)

                # If more than one matching file is found, raise an error
                if matching_file_count > 1:
                    print("\n")
                    print(row['clade_name'])
                    print(genome)
                    print(matching_file_path)
                    raise ValueError("Multiple matching files found")

        # Check the result and print the matching file path if one is found
        if matching_file_count == 0:
            print(row['clade_name'])
            print(genome)
            raise ValueError("No matching files found")
        elif matching_file_count == 1:
            paths.append(matching_file_path)
            #print("Found:", matching_file_path)

    type(paths)
    # Join the strings with newline characters and write to the file
    with open(outfile, 'w') as file:
        file.write('\n'.join(paths))
        file.write('\n')
        paths = []

## Run mOTUpan

On NERSC

# -- STEP 07 -- Processing mOTUpan results

In [4]:
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage3/df_gtdb_species_data_filteredStage3.pkl', 'rb') as file: df_gtdb_species_data_filteredStage3 = pickle.load(file)
#df_gtdb_species_data_filteredStage3
list_of_all_valid_genomes_Stage3 = df_gtdb_species_data_filteredStage3['Clustered genomes retained Stage3'].explode().tolist()
len(list_of_all_valid_genomes_Stage3)

307919

In [6]:
list_of_all_valid_genomes_Stage3[:2]

['GB_GCA_000193895.2', 'GB_GCA_000194495.2']

In [135]:
importlib.reload(pg)

<module 'pangenome_utils' from '/global/cfs/cdirs/kbase/ke_prototype/mcashman/scratch_notebooks/pangenome_utils.py'>

In [14]:
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtered_stage2/df_gtdb_metadata_filteredStage2.pkl', 'rb') as file: df_gtdb_metadata_filteredStage2 = pickle.load(file)
df_gtdb_metadata_filteredStage2['clade_name']=df_gtdb_metadata_filteredStage2['gtdb_taxonomy_species']+"--"+df_gtdb_metadata_filteredStage2['gtdb_genome_representative']
df_gtdb_metadata_filteredStage2['clade_name'].replace(' ', '_', regex=True, inplace=True)

col_of_interest = ['accession','clade_name','checkm_completeness','checkm_marker_count','contig_count','gtdb_taxonomy_species']
df = df_gtdb_metadata_filteredStage2[df_gtdb_metadata_filteredStage2['accession'].isin(list_of_all_valid_genomes_Stage3)][col_of_interest]

grouped_df = df.groupby('clade_name').agg(
    checkm_average=pd.NamedAgg(column='checkm_completeness', aggfunc='mean'),
    checkm_median=pd.NamedAgg(column='checkm_completeness', aggfunc='median'),
    checkm_min=pd.NamedAgg(column='checkm_completeness', aggfunc='min'),
    checkm_max=pd.NamedAgg(column='checkm_completeness', aggfunc='max'),
    contig_average=pd.NamedAgg(column='contig_count', aggfunc='mean'),
    contig_median=pd.NamedAgg(column='contig_count', aggfunc='median'),
    contig_min=pd.NamedAgg(column='contig_count', aggfunc='min'),
    contig_max=pd.NamedAgg(column='contig_count', aggfunc='max')
    # Add more statistics columns as needed
).reset_index()

grouped_df

,clade_name,checkm_average,checkm_median,checkm_min,checkm_max,contig_average,contig_median,contig_min,contig_max
0,s__0-14-0-80-60-11_sp018897875--GB_GCA_0188978...,89.226667,89.870,87.35,90.46,540.666667,537.0,442,643
1,s__0-14-3-00-41-53_sp002780895--GB_GCA_0027808...,93.844286,96.590,85.09,96.59,200.285714,230.0,71,322
2,s__01-FULL-36-15b_sp001782035--GB_GCA_001782035.1,60.890000,60.890,60.89,60.89,7.000000,7.0,5,9
3,s__01-FULL-44-24b_sp001793235--GB_GCA_001793235.1,56.105000,56.105,55.61,56.60,30.500000,30.5,29,32
4,s__01-FULL-45-10b_sp001804205--GB_GCA_001804205.1,88.375000,88.375,85.42,91.33,246.500000,246.5,202,291
...,...,...,...,...,...,...,...,...,...
27698,s__Zunongwangia_profunda--RS_GCF_000023465.1,87.163077,88.100,57.10,99.62,387.153846,423.0,1,742
27699,s__Zwartia_sp903958565--GB_GCA_903958565.1,97.966000,98.340,94.67,99.64,454.000000,417.0,407,587
27700,s__Zymobacter_palmae--RS_GCF_003610015.1,95.200000,95.200,95.19,95.21,36.500000,36.5,2,71
27701,s__Zymomonas_mobilis--RS_GCF_000175255.2,99.747308,100.000,94.11,100.00,18.846154,5.0,1,249


In [16]:
with open('/global/cfs/cdirs/kbase/ke_prototype/mcashman/analysis/dataframes/filtering_stage3/genome_list_A_SetOf58GPUclades.txt', 'r') as file:
    string_list = [line.strip() for line in file]
    
grouped_df[grouped_df['clade_name'].isin(string_list)]

,clade_name,checkm_average,checkm_median,checkm_min,checkm_max,contig_average,contig_median,contig_min,contig_max
765,s__Acinetobacter_baumannii--RS_GCF_009759685.1,99.821837,99.98,57.57,100.00,138.579058,97.0,1,1917
2461,s__Bacillus_A_bombysepticus--RS_GCF_006384875.1,98.933842,99.18,54.46,99.69,183.734727,123.5,1,1634
2551,s__Bacillus_subtilis--RS_GCF_000009045.1,99.592218,99.81,74.14,100.00,43.341682,14.0,1,1218
2556,s__Bacillus_velezensis--RS_GCF_001461825.1,99.618485,99.67,86.05,100.00,36.077280,4.0,1,1898
2778,s__Bifidobacterium_longum--RS_GCF_000196555.1,99.271630,100.00,66.59,100.00,65.646296,45.5,1,975
3064,s__Bordetella_pertussis--RS_GCF_000306945.1,99.534642,100.00,93.11,100.00,194.961727,1.0,1,1366
3342,s__Brucella_melitensis--RS_GCF_000007125.1,99.095995,99.45,87.85,100.00,32.812796,25.0,1,722
3409,s__Burkholderia_cenocepacia--RS_GCF_900446215.1,99.225651,99.73,92.74,99.87,160.485207,138.0,3,1996
3423,s__Burkholderia_mallei--RS_GCF_000011705.1,99.859408,100.00,95.25,100.00,128.373349,104.0,1,1141
3426,s__Burkholderia_multivorans--RS_GCF_000959525.1,99.853292,100.00,95.95,100.00,103.606436,83.5,2,1721


In [ ]:
clade = "s__Streptomyces_griseoviridis_A--RS_GCF_005222485.1"
output_locations = ['/global/cfs/cdirs/kbase/ke_prototype/mcashman/motupan/motupan-batch/scripts/14_ANIthread99pt9999/out_sizeLT5','/global/cfs/cdirs/kbase/ke_prototype/mcashman/motupan/motupan-batch/scripts/14_ANIthread99pt9999/out_sizeLT500','/global/cfs/cdirs/kbase/ke_prototype/mcashman/motupan/motupan-batch/scripts/14_ANIthread99pt9999/out_sizeGTE500']

out_file=pg.fetch_motupan_json_output_filepath(clade,output_locations)
out_file

df_temp = pd.read_json(out_file)
df_temp = df_temp.T

#df_temp.iloc[0]['aux_genome']
#df_temp['gene_clusterss'][df_temp['gene_clusterss'].keys()[0]]['aa']
df_temp['gene_clusterss'][df_temp['gene_clusterss'].keys()[0]]['genome']

In [1]:
import boto3
import yaml
from src.logger import setup_logger
from src.minio_uploader import upload_to_minio

# Set up logger
pipeline_name = "pangenome"
target_table = "genome"
schema = "pangenome"

logger = setup_logger(
    pipeline_name=pipeline_name,
    target_table=target_table,
    schema=schema
)

logger.info("Notebook started")

from config.config_loader import ConfigLoader

# Path to config file in MinIO
config_path = "genome"

# Load config
loader = ConfigLoader(config_path, logger)
loader.load_and_validate()

# Start Spark
from src.spark_session import start_spark_session
spark = start_spark_session(logger=logger)

{"time": "2025-07-10 15:40:43,963", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "2161914337", "msg": "Notebook started"}
{"time": "2025-07-10 15:40:43,976", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "ConfigLoader initialized for target table: genome"}
{"time": "2025-07-10 15:40:43,976", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Resolved config path: s3a://cdm-lake/config-json/genome.json"}
{"time": "2025-07-10 15:40:43,978", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Loading config from MinIO: bucket=cdm-lake, key=config-json/genome.json"}
{"time": "2025-07-10 15:40:44,065", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "config_loader", "msg": "Config loaded succes

In [2]:

# Validate input files
from src.input_file_validator import validate_input_files
validate_input_files(loader, logger)


# Check schema headers
from src.validate_schema import validate_schema_against_file
validate_schema_against_file(loader, logger)


# Run validations
from src.run_validations import run_validations_from_config
run_validations_from_config(loader, logger)


# Referential integrity
from src.run_referential_integrity_checks import run_referential_integrity_checks
run_referential_integrity_checks(spark, loader, logger)



{"time": "2025-07-10 15:40:51,040", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "config_loader", "msg": "Input files: [{'source': 'GTDB', 'file_path': 's3a://cdm-lake/bronze/gtdb/sp_clusters_r214.tsv', 'file_type': 'tsv', 'delimiter': '\t', 'ignore_first_line': 'no'}, {'source': 'NCBI', 'file_path': 's3a://cdm-lake/bronze/ncbi/assembly_summary_genbank.txt', 'file_type': 'tsv', 'delimiter': '\t', 'ignore_first_line': 'yes'}]"}
{"time": "2025-07-10 15:40:51,041", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "input_file_validator", "msg": "Starting input file validation..."}
Validating input files...

{"time": "2025-07-10 15:40:51,046", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "input_file_validator", "msg": "Checking file: s3a://cdm-lake/bronze/gtdb/sp_clusters_r214.tsv"}
🔍 Checking: s3a://cdm-lake/bronze/gtdb/sp_clusters_r214.tsv
{"time"

{"time": "2025-07-10 15:41:52,497", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "run_validations", "msg": "Delimiter: 	, Ignore first line: False"}
{"time": "2025-07-10 15:41:52,498", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "config_loader", "msg": "Validations: [{'column': 'genome_id', 'validation_type': 'not_null', 'error_message': 'Missing genome_id'}, {'column': 'genome_id', 'validation_type': 'regex_match', 'pattern': '^[A-Za-z0-9_.-]+$', 'error_message': 'Invalid genome_id format'}, {'column': 'gtdb_taxonomy_id', 'validation_type': 'not_null', 'error_message': 'Missing taxonomy ID'}, {'column': 'gtdb_species_clade_id', 'validation_type': 'not_null', 'error_message': 'Missing species clade ID'}]"}
{"time": "2025-07-10 15:41:52,498", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "run_validations", "msg": "Validation rules: [{'colu

{"time": "2025-07-10 15:42:19,392", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "ERROR", "module": "run_referential_integrity_checks", "msg": "Referential integrity violations written to Delta table: pangenome.genome_errors at s3a://cdm-lake/logs/errors/genome"}

🚨 All violations logged to: pangenome.genome_errors
{"time": "2025-07-10 15:42:19,394", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "WARNING", "module": "run_referential_integrity_checks", "msg": "Referential integrity check(s) failed."}

❌ Some referential integrity checks failed.


False

In [3]:
# Great Expectations
from src.run_great_expectations_validations import run_great_expectations_validation
run_great_expectations_validation(spark, loader, logger)


# Upload log file to MinIO
upload_to_minio(logger.log_file_path)

{"time": "2025-07-10 15:42:20,910", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "config_loader", "msg": "Target table: pangenome.genome"}
{"time": "2025-07-10 15:42:20,911", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Starting Great Expectations validation for table: pangenome.genome with suite: default_suite"}
{"time": "2025-07-10 15:42:20,965", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Loaded Spark table: pangenome.genome"}
{"time": "2025-07-10 15:42:21,126", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Great Expectations context initialized."}
{"time": "2025-07-10 15:42:21,151", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "lev

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{"time": "2025-07-10 15:42:22,279", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Applied GE expectation: expect_column_values_to_not_be_null with args {'column': 'genome_id'} → result: True"}


Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

{"time": "2025-07-10 15:42:22,639", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Applied GE expectation: expect_column_values_to_match_regex with args {'column': 'genome_id', 'regex': '^[A-Za-z0-9_.-]+$'} → result: True"}


...

{"time": "2025-07-10 15:42:23,086", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Applied GE expectation: expect_column_to_exist with args {'column': 'sample_id'} → result: False"}
{"time": "2025-07-10 15:42:23,119", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Expectations saved to suite."}


Calculating Metrics:   0%|          | 0/20 [00:00<?, ?it/s]

{"time": "2025-07-10 15:42:23,667", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Validation run completed."}
{"time": "2025-07-10 15:42:23,672", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "DEBUG", "module": "run_great_expectations_validations", "msg": "Validation summary: {'success': False, 'successful_expectations': 5, 'unsuccessful_expectations': 1, 'success_percent': 83.33333333333334}"}
{"time": "2025-07-10 15:42:25,809", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Data Docs built after validation."}


Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]

{"time": "2025-07-10 15:42:27,578", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Checkpoint executed."}
{"time": "2025-07-10 15:42:29,541", "pipeline": "pangenome", "schema": "pangenome", "table": "genome", "level": "INFO", "module": "run_great_expectations_validations", "msg": "Data Docs rebuilt with checkpoint results."}
✅ GE validation and checkpoint complete. Data Docs generated.
✅ Uploaded log to MinIO at: s3://cdm-lake/logs/pangenome/pipeline_run_20250710_154043.log
